In [ ]:
import torch
import numpy as np 
import os
import pandas as pd
import pickle
from dataloader import MeltomeUnirepDataset
import models

In [ ]:
# creating dataloader
URdir = "../datasets/UniRep_Vecs"
dict_path = "../datasets/protID2MT.p"
MUData = MeltomeUnirepDataset(URdir=URdir, dictPath=dict_path)

In [ ]:
import torch.optim as optimizer
import torch.nn as nn

# creating a classifier
model = models.SimpleNN(h_units=64)

# defining a loss function and optimizer
loss_fn = nn.MSELoss()
adam =  optimizer.Adam(model.parameters(), lr=0.0003)

In [ ]:
# determining if able to use gpu

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# training the classifier
model.to(device)

for epoch in range(2):
    running_loss = 0.0
    for i in range(len(MUData)):
        data = MUData[i]
        inputs, label = data["vec"].to(device), data["meltingTemp"].to(device)

        adam.zero_grad()

        output = model(inputs)
        loss = loss_fn(output, label)
        loss.backward()
        adam.step()

        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
